In [ ]:
# Gerekli kütüphaneleri yükleme
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener matplotlib openpyxl TvDatafeed

import pandas as pd
import matplotlib.pyplot as plt
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import Query, Column

# TradingView API bağlantısı
tv = TvDatafeed()

piyasa= 'turkey'
#piyasa= 'germany'
#piyasa= 'america'
options = ['stock']  #['stock', 'fund','dr']

#Burada tupple dönüyor
HisseAd_Sorgu = (Query().set_markets(f'{piyasa}')
                .select('name', 'type')
                .where(Column('type').isin(options) #(['stock', 'fund'])
                    )
                .limit(20000) #deault değeri 50 olduğu için hepsini getirmiyor.Onu aşmak için yüksek değer koydum.
                .order_by('name')
                .get_scanner_data())[1]

# DataFrame'e dönüştür
HisseAd = pd.DataFrame(HisseAd_Sorgu, columns=['ticker', 'name', 'type'])
#len(HisseAd)
#HisseAd.to_csv('HisseAd.csv')


# Piyasa bazında filtreleme
if piyasa == 'america':
    # Amerika için NYSE veya NASDAQ ile başlayanları seç
    filtered_HisseAd = HisseAd[HisseAd['ticker'].str.startswith(('NYSE', 'NASDAQ'))]
elif piyasa == 'turkey':
    # Turkey için herhangi bir filtreleme uygulanmaz
    filtered_HisseAd = HisseAd
elif piyasa == 'germany':
    # Germany için özel filtreleme (örneğin XETRA ile başlayanlar)
    filtered_HisseAd = HisseAd[HisseAd['ticker'].str.startswith(('GETTEX', 'TRADEGATE', 'XETRA'))]
else:
    # Diğer piyasalar için de benzer filtrelemeler eklenebilir
    filtered_HisseAd = HisseAd

# Listeye dönüştür
Hisseler = filtered_HisseAd['ticker'].tolist()
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

In [7]:
# Kullanıcıdan zaman dilimini seçme
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("\nLütfen bir zaman dilimi seçin (rakam girilecek):")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choice = input("Zaman dilimini seçin (1-9): ")
    interval_name, interval = intervals.get(choice, ('4 Saat', Interval.in_4_hour))
    return interval_name, interval

# Hacim spike kontrol fonksiyonu (Normalize veriler için)
def volume_spike_check(normalized_data, spike_threshold=2.5):
    # 10 periyotluk hacim ortalaması ekleme
    normalized_data['Volume_Avg'] = normalized_data['volume'].rolling(window=10).mean()

    # Son bar
    last_bar = normalized_data.iloc[-1]
    second_last_bar = normalized_data.iloc[-2]

    # Hacim spike kontrolü
    last_bar_spike = last_bar['volume'] > (last_bar['Volume_Avg'] * spike_threshold)

    # Fiyat hareketi analizi
    if last_bar_spike:
        if last_bar['close'] > second_last_bar['close']:
            return 'Pozitif Spike'
        elif last_bar['close'] < second_last_bar['close']:
            return 'Negatif Spike'
        else:
            return 'Yatay'
    return 'No Spike'

# Normalize edilmiş verilerle hacim spike taraması
def normalized_volume_spike_scan(Hisseler, interval):
    print("XU100 verileri çekiliyor...")
    # XU100 verilerini alma
    xu100_data = tv.get_hist(symbol='XU100', exchange='BIST', interval=interval, n_bars=100)
    xu100_data = xu100_data.reset_index()

    print("Hisse taraması başlatıldı...\n")
    positive_spike_list = []
    negative_spike_list = []

    for hisse in Hisseler:
        try:
            # Hisse verisini alma
            data = tv.get_hist(symbol=hisse, exchange='BIST', interval=interval, n_bars=100)
            data = data.reset_index()

            # Normalize edilmiş veriler oluşturma
            normalized_data = data.copy()
            for column in ['open', 'high', 'low', 'close', 'volume']:
                normalized_data[column] = 100 * (data[column] / xu100_data[column])

            # Hacim spike kontrolü
            spike_status = volume_spike_check(normalized_data)

            # Sonuçları listeye ekleme
            if spike_status == 'Pozitif Spike':
                print(f"{hisse} - Durum: Pozitif Spike")
                positive_spike_list.append(f"{hisse} - Pozitif Spike")
            elif spike_status == 'Negatif Spike':
                print(f"{hisse} - Durum: Negatif Spike")
                negative_spike_list.append(f"{hisse} - Negatif Spike")

        except Exception as e:
            print(f"{hisse} için veri alınırken hata oluştu: {e}")

    return positive_spike_list, negative_spike_list

# Kullanıcıdan zaman dilimi seçimi al
interval_name, interval = get_interval_choice()

# Taramayı başlat
positive_spike_results, negative_spike_results = normalized_volume_spike_scan(Hisseler, interval)

# Tarama sonuçlarını göster
if positive_spike_results or negative_spike_results:
    print("\nPozitif Volume Spike Tespit Edilen Hisseler:")
    for result in positive_spike_results:
        print(result)

    print("\nNegatif Volume Spike Tespit Edilen Hisseler:")
    for result in negative_spike_results:
        print(result)
else:
    print("Volume spike tespit edilen hisse bulunamadı.")



Lütfen bir zaman dilimi seçin (rakam girilecek):
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Zaman dilimini seçin (1-9): 7
XU100 verileri çekiliyor...
Hisse taraması başlatıldı...

AVOD - Durum: Pozitif Spike
EDIP - Durum: Negatif Spike
KARTN - Durum: Negatif Spike
KIMMR - Durum: Pozitif Spike
KRDMB - Durum: Pozitif Spike

Pozitif Volume Spike Tespit Edilen Hisseler:
AVOD - Pozitif Spike
KIMMR - Pozitif Spike
KRDMB - Pozitif Spike

Negatif Volume Spike Tespit Edilen Hisseler:
EDIP - Negatif Spike
KARTN - Negatif Spike


Son işlem hacmi, 10 periyotluk ortalama hacmin 2.5 katından fazlaysa spike olarak değerlendiriliyor
Bu eşik değeri (2.5) değiştirilebilir


Spike Sınıflandırması:


Pozitif Spike: Hacim artışı + Fiyat artışı
Negatif Spike: Hacim artışı + Fiyat düşüşü
Yatay: Hacim artışı + Fiyat değişimi yok


Zaman Dilimleri:


15dk'dan aylık periyoda kadar farklı zaman dilimlerinde analiz yapılabiliyor
Her zaman dilimi için ayrı spike karakteristikleri olabilir


Kullanım Alanları:


Anormal hacim hareketlerinin tespiti
Kurumsal alım-satımların tespiti
Manipülatif işlemlerin tespiti
Trend değişim sinyallerinin yakalanması


Avantajları:


Endeksten arındırılmış analiz
Farklı zaman dilimlerinde çalışabilme
Otomatik tarama yapabilme
Hem pozitif hem negatif hareketleri tespit edebilme


Eksik Yönleri/Geliştirilebilecek Alanlar:


Volatilite ayarlaması eklenebilir
Farklı hacim indikatörleri eklenebilir
Geçmiş spike istatistikleri tutulabilir
Alım-satım sinyalleri üretilebilir
Grafik arayüzü eklenebilir

Bu analiz özellikle şu durumlarda faydalı olabilir:

Piyasa genelinden ayrışan hareketleri tespit etmek
Büyük hacimli işlemleri yakalamak
Olası trend değişimlerini önceden görmek
Manipülatif hareketleri tespit etmek

In [ ]:
chat gptden

In [38]:
from tvDatafeed import TvDatafeed, Interval
from typing import Tuple, List, Dict
import time
import pandas as pd

def initialize_tv():
    """TradingView bağlantısını başlatır."""
    try:
        tv = TvDatafeed()
        return tv
    except Exception as e:
        print(f"TradingView bağlantısı kurulamadı: {str(e)}")
        return None

def get_interval_choice():
    """Kullanıcıdan zaman dilimi seçimini alır."""
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("\nLütfen bir zaman dilimi seçin (rakam girilecek):")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    while True:
        choice = input("Zaman dilimini seçin (1-9): ")
        if choice in intervals:
            return intervals[choice]
        print("Geçersiz seçim! Lütfen 1-9 arası bir rakam girin.")

def get_market_data(tv, symbol: str, interval: Interval, retries: int = 3) -> pd.DataFrame:
    """Belirtilen sembol için veri çeker, hata durumunda yeniden dener."""
    symbols_to_try = [symbol, f'BIST:{symbol}']

    for symbol_format in symbols_to_try:
        for attempt in range(retries):
            try:
                data = tv.get_hist(
                    symbol=symbol_format,
                    exchange='BIST',
                    interval=interval,
                    n_bars=50  # Daha düşük bar sayısı yeterli olabilir
                )
                if data is not None:
                    return data.reset_index()

                print(f"Veri alınamadı: {symbol_format}, Deneme {attempt + 1}/{retries}")
                time.sleep(1)  # API throttling'i önlemek için bekleme

            except Exception as e:
                print(f"Hata oluştu ({symbol_format}): {str(e)}")
                if attempt < retries - 1:
                    time.sleep(1)
                continue

    raise ValueError(f"{symbol} için veri alınamıyor.")

def volume_spike_check(normalized_data: pd.DataFrame, spike_threshold: float = 2.5) -> dict:
    """Hacim spike kontrolü yapar ve detaylı sonuç döndürür."""
    try:
        # 10 periyotluk hacim ortalaması
        normalized_data['Volume_Avg'] = normalized_data['volume'].rolling(window=10).mean()

        last_bar = normalized_data.iloc[-1]
        second_last_bar = normalized_data.iloc[-2]

        # Hacim ve fiyat analizi
        volume_ratio = last_bar['volume'] / last_bar['Volume_Avg']
        price_change = ((last_bar['close'] - second_last_bar['close']) / second_last_bar['close']) * 100

        # Spike kontrolü
        if volume_ratio > spike_threshold:
            if last_bar['close'] > second_last_bar['close']:
                spike_type = 'Pozitif Spike'
            elif last_bar['close'] < second_last_bar['close']:
                spike_type = 'Negatif Spike'
            else:
                spike_type = 'Yatay'

            return {
                'type': spike_type,
                'volume_ratio': volume_ratio,
                'price_change': price_change
            }

        return {'type': 'No Spike'}

    except Exception as e:
        print(f"Spike kontrolünde hata: {str(e)}")
        return {'type': 'Error'}

def check_ladder_volume(normalized_data: pd.DataFrame, steps: int = 3) -> bool:
    """Merdiven hacim oluşumunu kontrol eder."""
    try:
        volumes = normalized_data['volume'].iloc[-steps:]
        return all(x < y for x, y in zip(volumes, volumes[1:]))
    except Exception as e:
        print(f"Merdiven hacim kontrolünde hata: {str(e)}")
        return False

def normalized_volume_spike_scan(tv, hisseler: List[str], interval: Interval) -> Tuple[List[dict], List[dict]]:
    """Normalize edilmiş verilerle hacim spike taraması yapar."""
    print("XU100 verileri çekiliyor...")
    try:
        xu100_data = get_market_data(tv, 'XU100', interval)
    except Exception as e:
        print(f"XU100 verileri alınamadı: {str(e)}")
        return [], []

    print("Hisse taraması başlatıldı...\n")
    positive_spikes = []
    negative_spikes = []

    for hisse in hisseler:
        try:
            # Hisse verisini alma
            data = get_market_data(tv, hisse, interval)

            # Normalize edilmiş veriler oluşturma
            normalized_data = data.copy()
            for column in ['open', 'high', 'low', 'close', 'volume']:
                normalized_data[column] = 100 * (data[column] / xu100_data[column])

            # Hacim spike kontrolü
            spike_info = volume_spike_check(normalized_data)
            ladder_volume = check_ladder_volume(normalized_data)

            # Sonuçları kaydetme
            if spike_info['type'] in ['Pozitif Spike', 'Negatif Spike']:
                result = {
                    'symbol': hisse,
                    'type': spike_info['type'],
                    'volume_ratio': spike_info.get('volume_ratio', 0),
                    'price_change': spike_info.get('price_change', 0),
                    'ladder_volume': ladder_volume
                }

                if spike_info['type'] == 'Pozitif Spike':
                    positive_spikes.append(result)
                    print(f"{hisse} - Pozitif Spike (Hacim: {spike_info['volume_ratio']:.2f}x, "
                          f"Fiyat: {spike_info['price_change']:.2f}%, Merdiven Hacim: {ladder_volume})")
                else:
                    negative_spikes.append(result)
                    print(f"{hisse} - Negatif Spike (Hacim: {spike_info['volume_ratio']:.2f}x, "
                          f"Fiyat: {spike_info['price_change']:.2f}%, Merdiven Hacim: {ladder_volume})")

        except Exception as e:
            print(f"{hisse} için işlem yapılırken hata oluştu: {str(e)}")
            continue

    return positive_spikes, negative_spikes

def print_results(positive_spikes: List[dict], negative_spikes: List[dict]):
    """Tarama sonuçlarını formatlar ve gösterir."""
    if not (positive_spikes or negative_spikes):
        print("\nVolume spike tespit edilen hisse bulunamadı.")
        return

    if positive_spikes:
        print("\nPozitif Volume Spike Tespit Edilen Hisseler:")
        for spike in sorted(positive_spikes, key=lambda x: x['volume_ratio'], reverse=True):
            print(f"{spike['symbol']} - Hacim: {spike['volume_ratio']:.2f}x, "
                  f"Fiyat Değişimi: {spike['price_change']:.2f}%, Merdiven Hacim: {spike['ladder_volume']}")

    if negative_spikes:
        print("\nNegatif Volume Spike Tespit Edilen Hisseler:")
        for spike in sorted(negative_spikes, key=lambda x: x['volume_ratio'], reverse=True):
            print(f"{spike['symbol']} - Hacim: {spike['volume_ratio']:.2f}x, "
                  f"Fiyat Değişimi: {spike['price_change']:.2f}%, Merdiven Hacim: {spike['ladder_volume']}")

def main():
    """Ana program akışı."""
    # TradingView bağlantısını başlat
    tv = initialize_tv()
    if tv is None:
        print("Program sonlandırılıyor...")
        return

    # Kullanıcıdan zaman dilimi seçimini al
    interval_name, interval = get_interval_choice()

    # Hisse listesi
    hisseler =Hisseler

    # Taramayı başlat
    positive_spikes, negative_spikes = normalized_volume_spike_scan(tv, hisseler, interval)

    # Sonuçları göster
    print_results(positive_spikes, negative_spikes)

if __name__ == "__main__":
    main()



Lütfen bir zaman dilimi seçin (rakam girilecek):
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Zaman dilimini seçin (1-9): 7
XU100 verileri çekiliyor...
Hisse taraması başlatıldı...

AVOD - Pozitif Spike (Hacim: 4.24x, Fiyat: 3.99%, Merdiven Hacim: True)
BIGCH - Negatif Spike (Hacim: 3.10x, Fiyat: -1.02%, Merdiven Hacim: False)
DYOBY - Pozitif Spike (Hacim: 3.11x, Fiyat: 4.25%, Merdiven Hacim: False)
EDIP - Negatif Spike (Hacim: 3.97x, Fiyat: -0.06%, Merdiven Hacim: True)
ENTRA - Pozitif Spike (Hacim: 2.54x, Fiyat: 5.90%, Merdiven Hacim: False)
GRTHO - Pozitif Spike (Hacim: 2.87x, Fiyat: 3.06%, Merdiven Hacim: True)
KARTN - Negatif Spike (Hacim: 3.88x, Fiyat: -7.26%, Merdiven Hacim: True)
KIMMR - Pozitif Spike (Hacim: 4.16x, Fiyat: 3.81%, Merdiven Hacim: True)
KNFRT - Pozitif Spike (Hacim: 3.35x, Fiyat: 6.15%, Merdiven Hacim: True)
KRDMB - Pozitif Spike (Hacim: 3.55x, Fiyat: 2.04%, Merdiven Hacim: False)
VANGD - Pozitif Spike (Haci